# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.


In [ ]:
from google.colab import userdata
import os
from openai import OpenAI

# 하드코딩 방식(권장)
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# 런타임 환경변수 지정 -> 다른 API에서는 작동 안되는 경우도 있음
# os.environ["OPENAI_API_KEY"] = userdata.get(OPENAI_API_KEY)
# client = OpenAI()

## 대화형 챗봇

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content':'너는 친절한 챗봇이야'},
        {'role': 'user', 'content': '안녕 나는 차은우야'},
        {'role': 'assistant', 'content': '안녕 나는 차은우야'}, # 이름 기억
        {'role': 'user', 'content': '잘 지냈어? 내이름 기억나?'}
    ]
)
print(response.choices[0].message.content)

안녕, 나는 잘 지내고 있어! 네 이름은 차은우야. 오늘은 어떤 이야기하고 싶어?


In [ ]:
# Multi-turn 대화 (<-> Single-turn)
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content':'너는 LLM 전문가야'},
        {'role': 'user', 'content': '안녕 나는 LLM 꿈나무 차은우야'},
        {'role': 'assistant', 'content': '안녕하세요, 차은우님! LLM(대규모 언어 모델)에 대한 관심이 크신 것 같군요. 어떤 점에 대해 이야기해보고 싶으신가요? LLM의 작동 원리, 활용 사례, 혹은 연구 방향 등에 대해 궁금한 점이 있다면 언제든지 질문해 주세요!'}, # 대화내용 기억
        {'role': 'user', 'content': 'Transformer 모델을 공부하고 싶어.'},
        {'role': 'assistant', 'content': """
        좋아요! Transformer 모델은 최근 몇 년 동안 자연어 처리(NLP) 분야에서 혁신을 가져온 중요한 아키텍처입니다. Transformer의 주요 구성 요소와 작동 방식을 간단히 설명해 드릴게요.

### 1. Transformer의 구조
Transformer 모델은 기본적으로 인코더(encoder)와 디코더(decoder)로 구성되어 있습니다. 대부분의 최신 모델(예: BERT, GPT)은 이 구조의 변형을 사용합니다.

- **인코더**: 입력 시퀀스를 처리하여 고차원 벡터 표현을 생성합니다.
- **디코더**: 인코더에서 생성한 벡터를 사용하여 출력 시퀀스를 생성합니다.

### 2. Attention Mechanism
Transformer의 핵심은 "Attention" 메커니즘입니다. 이는 모델이 입력 시퀀스의 각 단어가 출력 시퀀스의 특정 단어에 얼마나 중요한지를 동적으로 평가할 수 있게 해줍니다.

- **Self-Attention**: 입력 시퀀스 내의 단어가 다른 단어와 어떤 관계가 있는지를 평가합니다.
- **Multi-Head Attention**: 여러 개의 attention heads를 사용하여 다양한 표현을 동시에 학습합니다.

### 3. 포지셔널 인코딩
Transformer는 RNN과 달리 순서 정보를 직접적으로 처리하지 않기 때문에, 각 단어의 위치 정보를 보존하기 위해 포지셔널 인코딩을 추가합니다. 이 벡터는 각 단어의 위치를 나타내는 정보를 포함합니다.

### 4. 피드포워드 레이어와 Layer Normalization
인코더와 디코더의 각 layer는 self-attention 후에 피드포워드 신경망을 포함하고 있습니다. 그 외에도 각 layer에서는 Layer Normalization이 적용되어 학습을 안정화합니다.

### 5. 학습 및 손실 함수
Transformer는 주로 교사 강요(teacher forcing) 방법을 사용하여 학습됩니다. 손실 함수로는 일반적으로 크로스 엔트로피 손실을 사용합니다.

### 참고 자료
- "Attention is All You Need" (Vaswani et al., 2017): Transformer 아키텍처에 대한 최초의 논문입니다.
- Various online tutorials and courses on platforms like Coursera, edX, or YouTube can also provide a good introduction.

더 구체적인 질문이나 하시고 싶은 다른 주제가 있으시면 말씀해 주세요!
"""},
        {'role': 'user', 'content': '어렵다. 어텐션을 초등학생도 이해할 수 있게 설명해줘'},
        {'role': 'assistant', 'content': """
        물론이죠! 초등학생도 이해할 수 있도록 쉽게 설명해볼게요.

### 어텐션 설명하기

어텐션은 중요한 것에 집중하는 방법이에요. 예를 들어, 친구들과 함께 동물원에 갔다고 생각해볼까요? 많은 동물이 있지만, 여러분은 사자를 가장 보랑하고 싶어요. 여러분은 사자를 찾기 위해 다른 동물들보다 사자에게 더 많이 주목하게 되죠. 이게 바로 어텐션이에요!

#### 차근차근 설명해볼게요:

1. **여러 동물**: 당신이 봐야 할 정보들이에요. 예를 들어, 개, 고양이, 사자, 기린 등이 있어요.
2. **중요한 동물**: 당신에게는 사자가 가장 중요해요. 그래서 사자에게 주목하고, 다른 동물들은 잠깐 건너뛰고 싶을 수 있어요.
3. **어텐션 작용**: 모델이 정보를 처리할 때, 각 단어(혹은 문장)에 얼마나 집중할지를 정하는 것이 어텐션이에요. 중요한 것에 더 많은 집중을 하고, 덜 중요한 것에는 조금 덜 집중하게 돼요.

### 간단한 예시

만약 "사자는 매우 강한 동물이다."라는 문장을 이해하려고 할 때, 모델은 '사자'라는 단어에 더 집중하게 되고, '강한 동물'이라는 부분은 덜 집중하는 방법이에요. '사자'가 이 문장의 핵심이니까요.

결국 어텐션은 중요한 정보에 집중하고 덜 중요한 정보는 지나치는 방법이라는 거예요! 이렇게 하면 모델이 더 잘 이해하고, 더 나은 답변을 할 수 있게 돼요.

이해가 되었나요? 더 궁금한 점이나 다른 질문이 있으면 언제든지 물어봐주세요!
"""},
        {'role': 'user', 'content': '이제 이해가 되는 것 같아. transformer 관련 대화내용을 요약해서 markdown 문서로 정리해줘'}
    ]
)

print(response.choices[0].message.content)

물론이죠! Transformer 모델에 대한 대화 내용을 요약하여 Markdown 형식으로 정리해드릴게요.

```markdown
# Transformer 모델 개요

## 1. Transformer 구조
- **인코더 (Encoder)**: 입력 시퀀스를 처리해 고차원 벡터로 변환.
- **디코더 (Decoder)**: 인코더의 출력을 기반으로 시퀀스 생성.

## 2. Attention Mechanism
- **Self-Attention**: 입력 시퀀스 내 단어 간의 관계를 평가.
- **Multi-Head Attention**: 여러 개의 attention heads를 사용해 다양한 표현 학습.

### 어텐션 예시
- 여러 동물이 있는 동물원에서 '사자'를 찾는 것에 비유.
- 중요한 정보에 집중하고 덜 중요한 정보는 지나칠 수 있도록 돕는다.

## 3. 포지셔널 인코딩
- RNN과는 달리 순서 정보를 직접 처리하지 않기 때문에 단어의 위치 정보를 추가함.

## 4. 피드포워드 레이어와 Layer Normalization
- 각 layer 단위로 self-attention 후 피드포워드 신경망 포함.
- 학습 안정화를 위한 Layer Normalization 적용.

## 5. 학습 및 손실 함수
- 주로 교사 강요 (teacher forcing) 방식으로 학습.
- 손실 함수로 크로스 엔트로피 손실 사용.

## 참고 자료
- **논문**: "Attention is All You Need" (Vaswani et al., 2017)
- **온라인 자료**: Coursera, edX, YouTube 등 다양한 튜토리얼 및 강좌.

---

이 문서에서 다룬 내용을 참고하여, Transformer 모델에 대한 이해를 넓혀가세요!
```

이렇게 작성된 내용을 활용하실 수 있습니다! 추가적인 질문이나 요청이 있다면 언제든지 말씀해 주세요.


## 반복처리

In [ ]:
# 대화내용을 로깅하는 messages 배열
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이야'}
]

print('종료하려면, exit를 입력하세요...')
while True:
  # 사용자 입력
  user_input = input('User: ')
  if user_input.strip().lower() == 'exit':
    print('채팅을 종료합니다...')
    break

  # messages에 사용자 입력 추가
  messages.append({'role': 'user', 'content': user_input})
  # print(messages)

  # LLM 요청
  response = client.chat.completions.create(
      model='gpt-4.1',
      messages=messages,
      temperature=1,
      top_p=1
  )
  assistant_message = response.choices[0].message.content
  print(f'Assistant: {assistant_message}')

  # messages 챗봇 출력 추가
  messages.append({'role':'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕 나 차은우야
Assistant: 안녕하세요, 차은우 씨! 만나서 정말 반가워요 😊  
혹시 오늘 어떤 이야기를 나누고 싶으신가요? 앨범이나 연기, 일상 등 궁금하신 것 있으면 언제든 말씀해 주세요!
User: 이삭토스트 메뉴 알려줘
Assistant: 네! 이삭토스트는 한국에서 아주 인기 있는 토스트 체인점이에요. 매장마다 약간씩 메뉴가 다를 수 있지만, 대표적인 메뉴를 소개해 드릴게요.

### 이삭토스트 인기 메뉴
1. **햄치즈 토스트**  
   부드러운 빵에 햄, 치즈, 양배추, 스페셜 소스가 들어가요.

2. **베이컨 베스트**  
   베이컨, 계란, 양배추, 스위트 마요네즈, 치즈가 들어간 메뉴예요.

3. **불갈비 MVP**  
   불고기 소고기 패티와 신선한 야채, 치즈, 특제 소스가 조화를 이루죠.

4. **폴더 에그**  
   폭신한 에그 스크램블이 가득 들어간 담백한 메뉴예요.

5. **핫치킨 MVP**  
   매운 치킨과 야채, 특제 소스의 매콤하고 고소한 맛!

6. **더블치즈 포테이토**  
   부드러운 감자와 두 가지 치즈, 신선한 야채가 들어가있어요.

7. **베이컨 포테이토**  
   감자와 베이컨이 어우러져 담백하고 고소한 맛을 느낄 수 있어요.

### 추가 메뉴
- 콘치즈 토스트
- 피자 토스트
- 새우 토스트
- 스팸 토스트
- 김치 베이컨 등

### 음료
- 아이스 아메리카노
- 오렌지 주스
- 우유 등

계절 및 매장 지역에 따라 일부 메뉴가 다를 수 있으니, 방문 전 이삭토스트 공식 홈페이지나 앱에서 확인해보시는 것도 좋아요!  
혹시 더 궁금한 메뉴나 가격, 추천 메뉴가 궁금하시면 말씀해 주세요! 😃
User: 맘스터치 메뉴는?
Assistant: 물론이죠! 맘스터치는 버거와 치킨으로 유명한 한국의 패스트푸드 브랜드입니다. 주요 매장 인기 메뉴를 소개해드릴게요.

---

### 맘스터치 대표 메뉴

#### 🍔 **버거**
- **싸이

## Stream

In [ ]:
stream = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': '지금 stream 테스트할거야. 아주 긴 응답 메시지를 보내줘'}],
    stream=True
)
for chunk in stream:
  content = chunk.choices[0].delta.content
  if content is not None:
    print(chunk.choices[0].delta.content, end=' ')

 물 론 입니다 !  긴  응 답  메시 지를  작성 해 드 릴 게 요 .  아래 는  다양한  주 제 에  대한  내용 으로  구성 된  긴  텍 스트 입니다 .

 ---

 ###  기술 의  발전 과  사회 적  영향 

 21 세 기가  시작 되 면서  우리는  정보 통 신  기술 (I CT )의  급 격 한  발전 을  경험 하고  있습니다 .  이러한  변화 는  우리의  일 상 생활 ,  경제 ,  교육 ,  그리고  사회  전 반 에  걸 쳐  큰  영향을  미 치 고  있습니다 .  특히  스마트 폰 의  보 급 과  인터넷 의  확 산 은  사람 들  간 의  소 통  방식 과  정보 에  접근 하는  방 식을  혁 신 적으로  변화 시 켰 습니다 .

 ####   1 .  정보 의  접근 성 

 정보  접근 성이  향 상 됨 에  따라 ,  우리는  언제  어디 서 나  필요한  정보를  얻 을  수  있는  시대 에  살 고  있습니다 .  구 글 ,  유 튜 브 와  같은  플랫폼 을  통해  사람들이  직접  정보를  검색 하고  공유 할  수  있게  되었 으며 ,  이는  개인 의  지 식  기반 을  넓 히 는  데  기 여 하고  있습니다 .  그러나  이러한  변화 는  또한  가 짜  뉴스 와  정보 의  왜 곡  문제 를  가져 오 기도  했 습니다 .

 ####   2 .  일 자리 의  변화 

 기 술 의  발전 으로  인해  많은  전 통 적인  일 자 리가  자동 화 되고  있습니다 .  제조 업 에서 의  로 봇  사용 은  생산 성을  높 이는  한 편 ,  단 순  노동 자 들의  일 자 리를  위 협 하고  있습니다 .  하지만  동시에  새로운  기술 을  다 루 는  직 업 도  생 겨 나 고  있으며 ,  데이터  분석 가 ,  인 공지 능 (A I )  전문가 ,  디 지털  마 케팅  전문가 와  같은  새로운  분야 는  강 력 한  수 요 를  보 이고  있습니다 .

 ####   3 

In [ ]:
# stream 버전
# 대화내용을 로깅하는 messages 배열
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이야'}
]

print('종료하려면, exit를 입력하세요...')
while True:
  # 사용자 입력
  user_input = input('User: ')

  if user_input.strip().lower() == 'exit':
    print('채팅을 종료합니다...')
    break

  # messages에 사용자 입력 추가
  messages.append({'role': 'user', 'content': user_input})
  # print(messages)

  # LLM 요청
  stream = client.chat.completions.create(
      model='gpt-4.1',
      messages=messages,
      temperature=1,
      top_p=1,
      stream=True
  )
  print(f'Assistant: ', end='')
  for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
      print(content, end='', flush=True) # 내부 buffer 사용하지 않음
  print()

  # messages 챗봇 출력 추가
  messages.append({'role':'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요...
User: 오늘 며칠이야?
Assistant: 오늘은 2023년 6월 21일이에요!  
혹시 다른 날짜를 원한다면 말씀해 주세요. 😊
User: exit
채팅을 종료합니다...


# Token counting

비용 = ((입력 토큰수 * 단가) + (출력 토큰수 * 단가)) * 월 서비스 호출 수

In [ ]:
!pip install tiktoken

In [ ]:
# 각 모델에 따른 토커나이저(인코딩 객체) 먼저 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o-mini')
gpt41 = tiktoken.get_encoding('cl100k_base') # 4.1 버전은 나온지 얼마 안되서 아직 준비가 안됨

print(gpt35) # <Encoding 'cl100k_base'>
print(gpt4o) # <Encoding 'o200k_base'>
print(gpt41)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>
<Encoding 'cl100k_base'>


In [ ]:
# 토큰 수 세기 -> 모델이 진화하면서 토큰을 경제적으로 처리할 수 있는 방법, 영어/다국어 격차를 줄이기 위한 노력을 하고 있음.
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다.')
encoded_gpt4o = gpt4o.encode('아버지가 방에 들어가신다.')

print(len(encoded_gpt35)) # 13
print(len(encoded_gpt4o)) # 10 -> 토큰 수 개선됨

13
10


## 토큰 비용 계산하기

In [ ]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_gpt4o = gpt4o.encode(text)
encoded_text_gpt41 = gpt41.encode(text)

print('gpt4o 토큰 수: ', len(encoded_text_gpt4o))
print('gpt41 토큰 수: ', len(encoded_text_gpt41))

gpt4o 토큰 수:  703
gpt41 토큰 수:  1215


In [ ]:
response_gpt4o = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content':'너는 똑부러지는 시사/경제 전문가로서, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=0.2 # 1보다 더 정확한 응답
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content':'너는 똑부러지는 시사/경제 전문가로서, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=0.2
)

output_gpt4o = response_gpt4o.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print('\n', 'gpt-4o 토큰수: ', len(output_gpt4o))
print('\n', 'gpt-4.1 토큰수: ', len(output_gpt41))


 gpt-4o 토큰수:  334

 gpt-4.1 토큰수:  653


In [44]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
  if model == 'gpt-4.1':
    encoding = tiktoken.get_encoding('cl100k_base')
  else:
    encoding = tiktoken.encoding_for_model(model)
  encoded = encoding.encode(text)
  return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
  input_tokens = count_tokens(input_text, model)
  output_tokens = count_tokens(output_text, model)

  # 모델별 단가 가져오기
  price = PRICING[model] # 입력/출력 단가 딕셔너리

  # 비용계산
  input_cost = (input_tokens / 1_000_000) * price['input']
  output_cost = (output_tokens / 1_000_000) * price['output']

  total_cost = (input_cost + output_cost) * num_service_call
  return total_cost

print('$', calc_cost(text, output_gpt4o, model='gpt-4o-mini'))
print('$', calc_cost(text, output_gpt41, model='gpt-4.1'))

$ 223.04999999999998
$ 7486.0


In [45]:
PRICING

{'gpt-4.1': {'input': 2.0, 'output': 8.0},
 'gpt-4.1-mini': {'input': 0.4, 'output': 1.6},
 'gpt-4.1-nano': {'input': 0.1, 'output': 0.4},
 'o1': {'input': 2.0, 'output': 8.0},
 'o3': {'input': 2.0, 'output': 8.0},
 'o4-mini': {'input': 1.1, 'output': 4.4},
 'gpt-4o': {'input': 2.5, 'output': 10.0},
 'gpt-4o-mini': {'input': 0.15, 'output': 0.6}}